In [1]:
#2018 HMDA Edit Testing File Generator
from collections import OrderedDict
import json
import pandas as pd
import random

#custom imports
import lar_constraints
import lar_generator



In [2]:
#2018 Filing Instruction Guide: https://www.consumerfinance.gov/data-research/hmda/static/for-filers/2018/2018-HMDA-FIG.pdf

use_cols = ['name', 'metDivName', 'countyFips', 'geoIdMsa', 'metDivFp', 'smallCounty', 'tracts']
cbsa_cols = ['name', 'metDivName', 'state', 'countyFips', 'county', 'tracts','geoIdMsa', 'metDivFp', 'smallCounty', 
             'stateCode', 'tractDecimal']
cbsas = pd.read_csv('../dependancies/tract_to_cbsa_2015.txt', usecols=use_cols, delimiter='|', 
                    header=None, names=cbsa_cols, dtype=str) #load tract to CBSA data from platform file
cbsas["tractFips"] = cbsas.countyFips + cbsas.tracts
counties = list(cbsas.countyFips)
tracts = list(cbsas.tractFips)



In [3]:
#load schemas for LAR and transmittal sheet
lar_schema_df = pd.DataFrame(json.load(open("../schemas/lar_schema.json", "r")))
ts_schema_df = pd.DataFrame(json.load(open("../schemas/ts_schema.json", "r")))


In [7]:

lar_gen = lar_generator.lar_gen(lar_schema_df, ts_schema_df, counties=counties, tracts=tracts) #instantiate generator
lar_const = lar_constraints.lar_constraints(counties=counties, tracts=tracts)#instantiate constraints

#test = lar_generator.lar_gen(lar_schema_df, ts_schema_df)
row = lar_gen.make_row() #create test row

#run constraint functions on row

file_length = 100 #set number of rows in test file

first = True
for i in range(file_length):
    print("creating row number: {num}".format(num=i))
    row = lar_gen.make_row() #create new row
    diff = [1] #initialize diff for loop
    while len(diff) >0:
        row_base = row.copy() #copy row to enable diff
        #apply constraint functions to LAR row
        for func in lar_const.constraint_funcs: 
            row = getattr(lar_const, func)(row)
            #convert initial and copied rows to sets for diff
            initial_row = set(row_base.items())
            changed_row = set(row.items())
            diff = (initial_row - changed_row)
#            print(len(initial_row), len(changed_row)) #check number of fields, should be 110
            #show readout of number of changes made to new LAR row
            print("changes from {func}: {vals}".format(func=str(func), vals=len(diff)))
    
    #create first row of dataframe
    if first:
        lar_frame = pd.DataFrame(row, index=[1])
        first = False
    #add additional rows to dataframe
    else:
        print("concating")
        new_lar = pd.DataFrame(row, index=[1])
        lar_frame = pd.concat([lar_frame, new_lar], axis=0)




creating row number: 0
changes from v612_const: 0
changes from v610_const: 1
changes from v613_const: 2
changes from v614_const: 2
changes from v615_const: 3
changes from v619_const: 3
changes from v622_const: 3
changes from v627_const: 3
changes from v628_const: 7
changes from v629_const: 8
changes from v630_const: 8
changes from v631_const: 12
changes from v632_const: 10
changes from v633_const: 10
changes from v634_const: 10
changes from v635_const: 12
changes from v636_const: 13
changes from v637_const: 13
changes from v638_const: 17
changes from v638_const: 17
changes from v640_const: 17
changes from v641_const: 17
changes from v643_const: 17
changes from v644_const: 18
changes from v645_const: 18
changes from v647_const: 19
changes from v648_const: 19
changes from v649_const: 19
changes from v650_const: 19
changes from v651_const: 19
changes from v652_const: 19
changes from v654_const: 20
changes from v655_const: 20
changes from v656_const: 21
changes from v657_const: 22
changes 

changes from v638_const: 8
changes from v638_const: 8
changes from v640_const: 8
changes from v641_const: 8
changes from v643_const: 8
changes from v644_const: 8
changes from v645_const: 8
changes from v647_const: 8
changes from v648_const: 8
changes from v649_const: 8
changes from v650_const: 8
changes from v651_const: 8
changes from v652_const: 8
changes from v654_const: 8
changes from v655_const: 8
changes from v656_const: 8
changes from v657_const: 9
changes from v658_const: 9
changes from v661_const: 9
changes from v662_const: 9
changes from v663_const: 11
changes from v664_const: 13
changes from v666_const: 13
changes from v667_const: 13
changes from v668_const: 13
changes from v669_const: 13
changes from v670_const: 14
changes from v671_const: 14
changes from v672_const: 14
changes from v673_const: 14
changes from v674_const: 14
changes from v675_const: 14
changes from v676_const: 14
changes from v677_const: 14
changes from v678_const: 14
changes from v679_const: 14
changes from

changes from v661_const: 2
changes from v662_const: 2
changes from v663_const: 2
changes from v664_const: 2
changes from v666_const: 2
changes from v667_const: 2
changes from v668_const: 2
changes from v669_const: 2
changes from v670_const: 2
changes from v671_const: 2
changes from v672_const: 2
changes from v673_const: 2
changes from v674_const: 2
changes from v675_const: 2
changes from v676_const: 2
changes from v677_const: 2
changes from v678_const: 2
changes from v679_const: 2
changes from v680_const: 2
changes from v681_const: 2
changes from v682_const: 2
changes from v688_const: 2
changes from v689_const: 2
changes from v690_const: 2
changes from v692_const: 2
changes from v693_const: 2
changes from v694_const: 2
changes from v696_const: 2
changes from v697_const: 2
changes from v698_const: 2
changes from v699_const: 2
changes from v700_const: 2
changes from v701_const: 2
changes from v702_const: 2
changes from v703_const: 2
changes from v704_const: 2
changes from v705_const: 2
c

changes from v668_const: 31
changes from v669_const: 34
changes from v670_const: 35
changes from v671_const: 35
changes from v672_const: 36
changes from v673_const: 37
changes from v674_const: 38
changes from v675_const: 39
changes from v676_const: 40
changes from v677_const: 41
changes from v678_const: 42
changes from v679_const: 43
changes from v680_const: 43
changes from v681_const: 43
changes from v682_const: 44
changes from v688_const: 44
changes from v689_const: 44
changes from v690_const: 45
changes from v692_const: 45
changes from v693_const: 45
changes from v694_const: 45
changes from v696_const: 48
changes from v697_const: 48
changes from v698_const: 48
changes from v699_const: 48
changes from v700_const: 50
changes from v701_const: 50
changes from v702_const: 50
changes from v703_const: 50
changes from v704_const: 50
changes from v705_const: 50
changes from v612_const: 0
changes from v610_const: 0
changes from v613_const: 0
changes from v614_const: 0
changes from v615_const:

changes from v614_const: 1
changes from v615_const: 1
changes from v619_const: 1
changes from v622_const: 1
changes from v627_const: 1
changes from v628_const: 5
changes from v629_const: 2
changes from v630_const: 2
changes from v631_const: 2
changes from v632_const: 2
changes from v633_const: 2
changes from v634_const: 2
changes from v635_const: 2
changes from v636_const: 2
changes from v637_const: 2
changes from v638_const: 2
changes from v638_const: 2
changes from v640_const: 2
changes from v641_const: 2
changes from v643_const: 2
changes from v644_const: 2
changes from v645_const: 2
changes from v647_const: 2
changes from v648_const: 2
changes from v649_const: 2
changes from v650_const: 2
changes from v651_const: 2
changes from v652_const: 2
changes from v654_const: 2
changes from v655_const: 2
changes from v656_const: 2
changes from v657_const: 2
changes from v658_const: 2
changes from v661_const: 2
changes from v662_const: 2
changes from v663_const: 2
changes from v664_const: 2
c

...

changes from v657_const: 27
changes from v658_const: 28
changes from v661_const: 28
changes from v662_const: 29
changes from v663_const: 29
changes from v664_const: 29
changes from v666_const: 29
changes from v667_const: 30
changes from v668_const: 30
changes from v669_const: 33
changes from v670_const: 33
changes from v671_const: 33
changes from v672_const: 34
changes from v673_const: 35
changes from v674_const: 36
changes from v675_const: 37
changes from v676_const: 38
changes from v677_const: 39
changes from v678_const: 40
changes from v679_const: 41
changes from v680_const: 41
changes from v681_const: 41
changes from v682_const: 42
changes from v688_const: 42
changes from v689_const: 42
changes from v690_const: 43
changes from v692_const: 43
changes from v693_const: 43
changes from v694_const: 44
changes from v696_const: 46
changes from v697_const: 46
changes from v698_const: 47
changes from v699_const: 47
changes from v700_const: 47
changes from v701_const: 47
changes from v702_co

changes from v705_const: 1
changes from v612_const: 0
changes from v610_const: 0
changes from v613_const: 1
changes from v614_const: 1
changes from v615_const: 1
changes from v619_const: 1
changes from v622_const: 1
changes from v627_const: 1
changes from v628_const: 5
changes from v629_const: 1
changes from v630_const: 1
changes from v631_const: 1
changes from v632_const: 1
changes from v633_const: 1
changes from v634_const: 1
changes from v635_const: 1
changes from v636_const: 1
changes from v637_const: 1
changes from v638_const: 1
changes from v638_const: 1
changes from v640_const: 1
changes from v641_const: 1
changes from v643_const: 1
changes from v644_const: 1
changes from v645_const: 1
changes from v647_const: 1
changes from v648_const: 1
changes from v649_const: 1
changes from v650_const: 1
changes from v651_const: 1
changes from v652_const: 1
changes from v654_const: 1
changes from v655_const: 1
changes from v656_const: 1
changes from v657_const: 1
changes from v658_const: 1
c

changes from v613_const: 0
changes from v614_const: 0
changes from v615_const: 0
changes from v619_const: 0
changes from v622_const: 0
changes from v627_const: 0
changes from v628_const: 0
changes from v629_const: 0
changes from v630_const: 0
changes from v631_const: 0
changes from v632_const: 0
changes from v633_const: 0
changes from v634_const: 0
changes from v635_const: 0
changes from v636_const: 0
changes from v637_const: 0
changes from v638_const: 0
changes from v638_const: 0
changes from v640_const: 0
changes from v641_const: 0
changes from v643_const: 0
changes from v644_const: 0
changes from v645_const: 0
changes from v647_const: 0
changes from v648_const: 0
changes from v649_const: 0
changes from v650_const: 0
changes from v651_const: 0
changes from v652_const: 0
changes from v654_const: 0
changes from v655_const: 0
changes from v656_const: 1
changes from v657_const: 1
changes from v658_const: 1
changes from v661_const: 1
changes from v662_const: 1
changes from v663_const: 1
c

changes from v676_const: 19
changes from v677_const: 19
changes from v678_const: 19
changes from v679_const: 19
changes from v680_const: 19
changes from v681_const: 19
changes from v682_const: 19
changes from v688_const: 19
changes from v689_const: 19
changes from v690_const: 19
changes from v692_const: 19
changes from v693_const: 18
changes from v694_const: 18
changes from v696_const: 20
changes from v697_const: 20
changes from v698_const: 20
changes from v699_const: 20
changes from v700_const: 20
changes from v701_const: 20
changes from v702_const: 20
changes from v703_const: 20
changes from v704_const: 20
changes from v705_const: 20
changes from v612_const: 0
changes from v610_const: 0
changes from v613_const: 0
changes from v614_const: 0
changes from v615_const: 0
changes from v619_const: 0
changes from v622_const: 0
changes from v627_const: 0
changes from v628_const: 0
changes from v629_const: 0
changes from v630_const: 0
changes from v631_const: 3
changes from v632_const: 0
chang

changes from v668_const: 0
changes from v669_const: 0
changes from v670_const: 0
changes from v671_const: 0
changes from v672_const: 0
changes from v673_const: 0
changes from v674_const: 0
changes from v675_const: 0
changes from v676_const: 0
changes from v677_const: 0
changes from v678_const: 0
changes from v679_const: 0
changes from v680_const: 0
changes from v681_const: 0
changes from v682_const: 0
changes from v688_const: 0
changes from v689_const: 0
changes from v690_const: 0
changes from v692_const: 0
changes from v693_const: 0
changes from v694_const: 0
changes from v696_const: 0
changes from v697_const: 0
changes from v698_const: 0
changes from v699_const: 0
changes from v700_const: 0
changes from v701_const: 0
changes from v702_const: 0
changes from v703_const: 0
changes from v704_const: 0
changes from v705_const: 0
concating
creating row number: 97
changes from v612_const: 1
changes from v610_const: 1
changes from v613_const: 2
changes from v614_const: 3
changes from v615_con

In [8]:
#check output of LAR generator
lar_frame

,record_id,lei,uli,app_date,loan_type,loan_purpose,preapproval,const_method,occ_type,loan_amount,...,aus_code_5,aus_result_1,aus_result_2,aus_result_3,aus_result_4,aus_result_5,aus_code_16,reverse_mortgage,open_end_credit,business_purpose
1,2,F7TFK7303TKN2N9DRPE3,F7TFK7303TKN2N9DRPE3KQ4TXWT5MW8F15AQRHKQYAXGM,NA,1,1,2,2,2,9439,...,,17,,,,,,1,1,1
1,2,BCG37NFXOA8MAKUYTZGU,BCG37NFXOA8MAKUYTZGUXDMDIEFDZBOJI1ILZN7QIX1QY,NA,2,1,2,2,3,9530,...,,17,,,,,,1,2,2
1,2,5MSL4KS4TB3BPLK3RCRN,5MSL4KS4TB3BPLK3RCRNX74H3M5I1TGPHHHOUYMDZJ71A,NA,1,1,2,2,2,6304,...,,17,,,,,,2,1,2
1,2,ADLEY8IPY3URZJ3J1KB6,ADLEY8IPY3URZJ3J1KB6UCH3D5FTRK2H57FTLCIPF0XA0,20180405,3,1,2,2,1,3266,...,8RF5BIXLS3P2V5VOLM4AZ8GYTCX5C99NKN5A8GV9202E07...,1,3,,,,,1,1,2
1,2,JPHZ9W7LNYTRBH76SL20,JPHZ9W7LNYTRBH76SL20B9HVJGHGV2ZJTD2VJL7EI97BD,NA,2,1,2,2,2,5498,...,,5,,,,,,2,1,1
1,2,7J9YO1LFWKDW8OYE0SPB,7J9YO1LFWKDW8OYE0SPBNPMOX2A5SNJLKC5JWXOPL90FU,20180413,4,1,2,2,2,9502,...,WCORIEMB1W2NU1MAYZ8TJ7XAH969H32MDKS7C1LQ3ADS86...,8,,,,,,1,2,2
1,2,FSE5JRTBHRZEBA7RIA5L,FSE5JRTBHRZEBA7RIA5LCAX1NF6MYH88ZSVA99KUJBWDV,20180522,4,1,2,2,1,6738,...,,7,,,,,,1,2,1
1,2,T4CGL8D8RDJ0K75ZL9U4,T4CGL8D8RDJ0K75ZL9U4MFJU4EY9FQ8RFYP76T1Z1285H,20180529,1,5,2,2,3,4005,...,,17,,,,,,1,2,2
1,2,CRIX5FSCODCRDCN3Y28P,CRIX5FSCODCRDCN3Y28P7O1GAD59GDLZTZ5Z0FKONT2MH,20180302,3,1,2,2,2,5637,...,,4,,,,,,1,1,2
1,2,JOGI0B7OTAK06WFVR18Y,JOGI0B7OTAK06WFVR18YNZADMVWRR0U7P563PXF0JI2Y2,20180707,3,1,2,2,2,4302,...,,5,,,,,,1,2,1


In [ ]:
#Quality and Macro field interrelationship constraints:


# mlo_id needs NA option




#set dummy values for TS row
ts_row_small = OrderedDict()

ts_row_small["record_id"]="1"
ts_row_small["inst_name"]="Ficus Bank"
ts_row_small["calendar_year"]=str(2018)
ts_row_small["calendar_quarter"]=str(TS_df.valid_vals[TS_df.field=="calendar_quarter"].iloc[0][0])
ts_row_small["contact_name"]="Mr. Smug Pockets"
ts_row_small["contact_tel"]="555-555-5555"
ts_row_small["contact_email"]="pockets@ficus.com"
ts_row_small["contact_street_address"]="1234 Ficus Lane"
ts_row_small["office_city"]="Ficusville"
ts_row_small["office_state"]="UT"
ts_row_small["office_zip"]="84096"
ts_row_small["federal_agency"]=str(TS_df.valid_vals[TS_df.field=="federal_agency"].iloc[0][0])
ts_row_small["lar_entries"]=str(lar_small)
ts_row_small["tax_id"]="01-0123456"
ts_row_small["lei"]="12345678901234567890"

#create TS medium row
ts_row_medium = ts_row_small.copy()
ts_row_medium["lar_entries"] = str(lar_medium)

#create TS large row
ts_row_large = ts_row_small.copy()
ts_row_large["lar_entries"] = str(lar_large)

